In [ ]:
!pip install tensorflow
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!nvidia-smi

Sat Mar 18 06:39:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Part 1 - Building the CNN
#importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
import matplotlib.pyplot as plt
import h5py

#Input Image Size
sz=300

In [ ]:
# Initialing the CNN
classifier = Sequential()

In [ ]:
# Step 1 - Convolutio Layer 
classifier.add(Convolution2D(32, (3,3) , input_shape = (sz, sz,3), activation = 'relu', padding='same'))

In [ ]:
#step 2 - Pooling
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [ ]:
# Adding second convolution layer
classifier.add(Convolution2D(32, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))

In [ ]:
#Adding 3rd Concolution Layer
classifier.add(Convolution2D(64, (3,3) , activation = 'relu', padding='same'))
classifier.add(MaxPooling2D(pool_size =(2,2)))


In [ ]:
#Step 3 - Flattening
classifier.add(Flatten())

In [ ]:
#Step 4 - Full Connection
classifier.add(Dense(256, activation = 'relu'))
classifier.add(Dropout(0.4))
classifier.add(Dense(36, activation = 'softmax')) # 36 Units are the value of no of Classes

In [ ]:
#Compiling The CNN
classifier.compile(
              optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
# Classifier Summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [ ]:
#Part 2 Fittting the CNN to the image
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [ ]:
#Dataset Store in Variable
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/train',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical')

test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/Data/test',
        target_size=(sz, sz),
        batch_size=32,
        color_mode='rgb',
        class_mode='categorical')

Found 7605 images belonging to 36 classes.
Found 3330 images belonging to 36 classes.


In [ ]:
#Model Initialize
epochs=25
model = classifier.fit_generator(
        training_set,
        steps_per_epoch=218,	#No of Image in Traing Set 7000/batch size(32)=218
        epochs=epochs, #Previous value is 25
        validation_data = test_set,
        validation_steps =54 	#No of Image in Test Set 1750/batch size(32)=54
      )

<ipython-input-16-0994f195a884>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model = classifier.fit_generator(


Epoch 1/25
237/237 [==============================] - 2239s 9s/step - loss: 1.7667 - accuracy: 0.4998 - val_loss: 1.4951 - val_accuracy: 0.6029
Epoch 2/25
237/237 [==============================] - 186s 786ms/step - loss: 0.4942 - accuracy: 0.8496 - val_loss: 1.5262 - val_accuracy: 0.6192
Epoch 3/25
237/237 [==============================] - 187s 789ms/step - loss: 0.3261 - accuracy: 0.9019 - val_loss: 1.3940 - val_accuracy: 0.6657
Epoch 4/25
237/237 [==============================] - 176s 744ms/step - loss: 0.2241 - accuracy: 0.9262 - val_loss: 1.3289 - val_accuracy: 0.7142
Epoch 5/25
237/237 [==============================] - 179s 754ms/step - loss: 0.1841 - accuracy: 0.9387 - val_loss: 1.2649 - val_accuracy: 0.7433
Epoch 6/25
237/237 [==============================] - 182s 765ms/step - loss: 0.1535 - accuracy: 0.9478 - val_loss: 1.4253 - val_accuracy: 0.7384
Epoch 7/25
237/237 [==============================] - 179s 755ms/step - loss: 0.1421 - accuracy: 0.9550 - val_loss: 1.5973 - v

In [ ]:
#Saving the model
classifier.save('/content/drive/MyDrive/Models/Trained_model.h5')

# model_json = classifier.to_json()
# with open("/content/drive/MyDrive/Data/model-bw1.json", "w") as json_file:
#     json_file.write(model_json)
print('Model Saved')

print(model.history.keys())

Model Saved
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
#Accuracy
model.history['accuracy']

[0.4998019337654114,
 0.8495972752571106,
 0.9018883109092712,
 0.926185131072998,
 0.9387297034263611,
 0.9478409886360168,
 0.9549716114997864,
 0.9643470048904419,
 0.9607817530632019,
 0.9693648219108582,
 0.9687045812606812,
 0.9720057845115662,
 0.9731942415237427,
 0.9749108552932739,
 0.9786081910133362,
 0.9807209968566895,
 0.9816453456878662,
 0.9791364073753357,
 0.9817773699760437,
 0.9813812375068665,
 0.9832298755645752,
 0.9840222001075745,
 0.9819093942642212,
 0.9813812375068665,
 0.9870592951774597]